<a href="https://colab.research.google.com/github/BobSheehan23/Bob_EquiLend_Models/blob/main/fredav.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from fredapi import Fred
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import requests
import time
from statsmodels.api import OLS, add_constant
from datetime import date

# =========================
# CONFIG & KEYS
# =========================
FRED_API_KEY = "7f8e44038ee69c4f78cf71873e85db16"                   # provided
ALPHAVANTAGE_API_KEY = "IOTZFZG01XK55BHI"                           # provided
START_DATE = "2015-01-01"                                           # adjust the analysis window
TITLE = "When Real Yields Rise but Gold Doesn’t Blink"
SUBTITLE = "A decoupling worth your attention—and a simple regime map to trade it."

# Branding palette
COLORS = {
    "blue":   "#0067db",  # primary
    "orange": "#ff8c42",  # medium-dark but bright orange
    "purple": "#6a0dad",  # deep purple
    "gray":   "#7a7a7a",  # medium-dark gray
}

plt.rcParams.update({
    "axes.edgecolor": "#333333",
    "axes.titleweight": "bold",
    "axes.labelcolor": "#333333",
    "font.size": 12,
    "figure.dpi": 150,
    "savefig.dpi": 180,
})

# Output paths
OUT = Path("lighthouse_outputs")
CH = OUT / "charts"
TB = OUT / "tables"
OUT.mkdir(exist_ok=True, parents=True)
CH.mkdir(exist_ok=True, parents=True)
TB.mkdir(exist_ok=True, parents=True)

# =========================
# HELPERS
# =========================
def monthly(series: pd.Series) -> pd.Series:
    """Coerce to month-end frequency with last available obs."""
    s = pd.Series(series).dropna()
    s.index = pd.to_datetime(s.index)
    return s.resample("M").last().ffill()

def clip(s: pd.Series, start=START_DATE) -> pd.Series:
    return s[s.index >= pd.to_datetime(start)]

def index_100(s: pd.Series) -> pd.Series:
    s = s.dropna()
    return s / s.iloc[0] * 100.0

def pct_change_12m(s: pd.Series) -> pd.Series:
    return s.pct_change(12) * 100.0

def monthly_returns(s: pd.Series) -> pd.Series:
    return s.pct_change(1).rename(s.name + "_ret")

def delta(s: pd.Series) -> pd.Series:
    """Simple first difference (e.g., monthly change in yields)."""
    return s.diff(1).rename(s.name + "_chg")

def roll_corr(a: pd.Series, b: pd.Series, window=12) -> pd.Series:
    return a.rolling(window).corr(b)

def ols_beta(y: pd.Series, x: pd.Series):
    """OLS beta of y on x with t-stat and n-obs."""
    df = pd.concat([y, x], axis=1).dropna()
    if len(df) < 12:
        return np.nan, np.nan, np.nan
    X = add_constant(df.iloc[:,1].values)
    model = OLS(df.iloc[:,0].values, X).fit()
    return float(model.params[1]), float(model.tvalues[1]), int(model.nobs)

def to_monthly_from_daily(df: pd.DataFrame, value_col: str) -> pd.Series:
    """Convert daily AV time series to monthly last."""
    s = df[value_col].copy()
    s.index = pd.to_datetime(s.index)
    s = s.sort_index().resample("M").last().ffill()
    return s

def av_get(symbol: str, function: str, **params) -> dict:
    """Generic Alpha Vantage GET with basic retry/backoff."""
    base = "https://www.alphavantage.co/query"
    payload = {"function": function, "apikey": ALPHAVANTAGE_API_KEY}
    payload.update(params)
    for i in range(3):
        r = requests.get(base, params=payload, timeout=30)
        if r.status_code == 200:
            data = r.json()
            # Handle throttling / note
            if any(k in data for k in ["Error Message", "Information", "Note"]):
                time.sleep(12 * (i + 1))
                continue
            return data
        time.sleep(3 * (i + 1))
    return {}

def av_fx_daily(from_symbol: str, to_symbol: str) -> pd.DataFrame:
    """FX daily series (e.g., XAUUSD if supported)."""
    data = av_get(function="FX_DAILY", from_symbol=from_symbol, to_symbol=to_symbol, outputsize="full")
    key = "Time Series FX (Daily)"
    if key not in data:
        return pd.DataFrame()
    df = pd.DataFrame(data[key]).T.rename(columns={
        "1. open":"open", "2. high":"high", "3. low":"low", "4. close":"close"
    }).apply(pd.to_numeric, errors="coerce")
    df.index.name = "date"
    return df

def av_equity_monthly(symbol: str) -> pd.DataFrame:
    """Monthly OHLC for ETFs/equities (e.g., GLD, UUP)"""
    data = av_get(function="TIME_SERIES_MONTHLY_ADJUSTED", symbol=symbol)
    key = "Monthly Adjusted Time Series"
    if key not in data:
        return pd.DataFrame()
    df = pd.DataFrame(data[key]).T.rename(columns={
        "1. open":"open", "2. high":"high", "3. low":"low", "4. close":"close",
        "5. adjusted close":"adj_close", "6. volume":"volume", "7. dividend amount":"dividend"
    }).apply(pd.to_numeric, errors="coerce")
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    return df

def safe_plot(series_list, labels, colors, title, ylab, fname, hline0=False):
    plt.figure(figsize=(9,5))
    for s, lab, col in zip(series_list, labels, colors):
        s.dropna().plot(label=lab, color=col)
    if hline0:
        plt.axhline(0, linestyle="--", color=COLORS["gray"])
    plt.title(title)
    plt.ylabel(ylab); plt.xlabel("")
    plt.legend()
    plt.tight_layout()
    plt.savefig(CH / fname)
    plt.close()

# =========================
# FETCH DATA — FRED first
# =========================
fred = Fred(api_key=FRED_API_KEY)

# Deep-dive backbone
dfii10 = clip(monthly(fred.get_series("DFII10")))                 # 10Y TIPS real yield (%)
usd_broad = clip(monthly(fred.get_series("DTWEXBGS")))            # Trade-Weighted USD: Broad
gold_fred = clip(monthly(fred.get_series("GOLDAMGBD228NLBM")))    # Gold (London AM USD)

# Dashboard
payems = clip(monthly(fred.get_series("PAYEMS")))                 # Nonfarm Payrolls (thousands)
unrate = clip(monthly(fred.get_series("UNRATE")))                 # Unemployment rate (%)
cpi    = clip(monthly(fred.get_series("CPIAUCSL")))               # CPI Index
core   = clip(monthly(fred.get_series("CPILFESL")))               # Core CPI Index
lei    = clip(monthly(fred.get_series("USSLIND")))                # Leading Index (Conference Board)
umich  = clip(monthly(fred.get_series("UMCSENT")))                # U. Michigan Sentiment

# =========================
# ALPHA VANTAGE — Proxies or fills
# Use GLD as market gold proxy & UUP as USD proxy (if you want market-tradable series)
# Also try XAUUSD via FX_DAILY (if AV supports the metal code in your account)
# =========================
# GLD monthly
gld_m = av_equity_monthly("GLD")
gld_m_close = gld_m["adj_close"] if not gld_m.empty else pd.Series(dtype=float)
gld_m_close.name = "GLD_adj_close"

# UUP monthly (Dollar proxy ETF)
uup_m = av_equity_monthly("UUP")
uup_m_close = uup_m["adj_close"] if not uup_m.empty else pd.Series(dtype=float)
uup_m_close.name = "UUP_adj_close"

# XAUUSD daily -> monthly (if available)
xauusd = av_fx_daily("XAU", "USD")
xauusd_m = to_monthly_from_daily(xauusd, "close") if not xauusd.empty else pd.Series(dtype=float)
xauusd_m.name = "XAUUSD"

# Trim AV proxies to START_DATE
if not gld_m_close.empty:
    gld_m_close = gld_m_close[gld_m_close.index >= pd.to_datetime(START_DATE)]
if not uup_m_close.empty:
    uup_m_close = uup_m_close[uup_m_close.index >= pd.to_datetime(START_DATE)]
if not xauusd_m.empty:
    xauusd_m = xauusd_m[xauusd_m.index >= pd.to_datetime(START_DATE)]

# =========================
# SERIES SELECTION (primary + proxies)
# =========================
gold_series = gold_fred.copy()
gold_label = "Gold (London AM, USD)"
if gold_series.dropna().empty and not gld_m_close.empty:
    gold_series = gld_m_close.copy()
    gold_label = "GLD (Adj Close, proxy)"
elif not xauusd_m.empty:
    # Option: blend or choose XAUUSD if you prefer spot-like
    # For now we keep FRED as primary; uncomment to switch:
    # gold_series = xauusd_m.copy()
    # gold_label = "XAUUSD (Alpha Vantage FX)"
    pass

usd_series = usd_broad.copy()
usd_label = "USD Broad Index (DTWEXBGS)"
if usd_series.dropna().empty and not uup_m_close.empty:
    usd_series = uup_m_close.copy()
    usd_label = "UUP (Adj Close, proxy)"

# =========================
# CALCULATIONS
# =========================
# Index for visuals
gold_idx = index_100(gold_series)
usd_idx  = index_100(usd_series)
dfii_idx = index_100(dfii10)

# Excess performance (12m)
gold_12 = pct_change_12m(gold_series)
usd_12  = pct_change_12m(usd_series)
excess  = (gold_12 - usd_12).dropna()
excess.name = "Gold minus USD (12m pp)"

# Rolling correlations (12m) with ΔReal Yield
gold_ret = monthly_returns(gold_series)
usd_ret  = monthly_returns(usd_series)
d_real   = delta(dfii10)

corr_gold_y = roll_corr(gold_ret, d_real, window=12).dropna()
corr_usd_y  = roll_corr(usd_ret,  d_real, window=12).dropna()

# Regimes: Level tertiles & 3m slope sign
p33, p67 = dfii10.quantile([0.33, 0.67])
level_regime = pd.Series(index=dfii10.index, dtype="object")
level_regime[dfii10 <= p33] = "Low"
level_regime[(dfii10 > p33) & (dfii10 <= p67)] = "Mid"
level_regime[dfii10 > p67] = "High"

def slope_3m(x: pd.Series) -> float:
    x = x.dropna()
    if len(x) < 3: return np.nan
    y = x.values
    t = np.arange(len(y))
    return np.polyfit(t, y, 1)[0]

rolling_slope = dfii10.rolling(3).apply(slope_3m, raw=False)
trend_regime = pd.Series(index=dfii10.index, dtype="object")
trend_regime[rolling_slope > 0]  = "Rising"
trend_regime[rolling_slope <= 0] = "Falling"

reg_df = pd.concat({
    "gold_ret": gold_ret,
    "usd_ret": usd_ret,
    "d_real": d_real,
    "level_regime": level_regime,
    "trend_regime": trend_regime
}, axis=1).dropna()

# Regime betas: returns ~ Δreal yield
rows = []
for asset in ["gold_ret","usd_ret"]:
    for lvl in ["Low","Mid","High"]:
        for tr in ["Falling","Rising"]:
            sub = reg_df[(reg_df["level_regime"]==lvl) & (reg_df["trend_regime"]==tr)]
            b, t, n = ols_beta(sub[asset], sub["d_real"])
            rows.append({
                "Asset": asset.replace("_ret","").upper(),
                "LevelRegime": lvl,
                "TrendRegime": tr,
                "Beta_vs_dRealYield": b,
                "t_stat": t,
                "Obs": n
            })
betas = pd.DataFrame(rows).sort_values(["Asset","LevelRegime","TrendRegime"])
betas.to_csv(TB / "regime_betas_vs_dRealYield.csv", index=False)

# Quick dashboard stats
def yoy(series):
    r = series.pct_change(12) * 100
    return float(r.dropna().iloc[-1]) if not r.dropna().empty else np.nan

dash_stats = {
    "payems_last": float(payems.dropna().iloc[-1]) if not payems.dropna().empty else np.nan,
    "unrate_last": float(unrate.dropna().iloc[-1]) if not unrate.dropna().empty else np.nan,
    "cpi_yoy": yoy(cpi),
    "core_yoy": yoy(core),
    "lei_last": float(lei.dropna().iloc[-1]) if not lei.dropna().empty else np.nan,
    "umich_last": float(umich.dropna().iloc[-1]) if not umich.dropna().empty else np.nan,
}

# =========================
# CHARTS
# =========================
safe_plot(
    [gold_idx, dfii_idx],
    [f"{gold_label} (indexed)", "10y TIPS Real Yield (indexed)"],
    [COLORS["blue"], COLORS["orange"]],
    "Real Yields vs Gold — Indexed to 100",
    "Index (start=100)",
    "deepdive_1_real_vs_gold_indexed.png"
)

safe_plot(
    [usd_idx, dfii_idx],
    [f"{usd_label} (indexed)", "10y TIPS Real Yield (indexed)"],
    [COLORS["blue"], COLORS["orange"]],
    "Real Yields vs Trade-Weighted USD — Indexed to 100",
    "Index (start=100)",
    "deepdive_2_real_vs_usd_indexed.png"
)

safe_plot(
    [excess],
    ["Gold minus USD (12m pp)"],
    [COLORS["blue"]],
    "12-Month Excess Performance: Gold minus USD",
    "pp",
    "deepdive_3_gold_minus_usd_12m.png",
    hline0=True
)

safe_plot(
    [corr_gold_y, corr_usd_y],
    ["Corr( Gold returns, ΔReal yield )", "Corr( USD returns, ΔReal yield )"],
    [COLORS["blue"], COLORS["purple"]],
    "Rolling 12-Month Correlations with ΔReal Yield",
    "Correlation",
    "deepdive_4_rolling_corrs.png",
    hline0=True
)

# Regime beta bars
for asset in ["GOLD","USD"]:
    sub = betas[betas["Asset"]==asset]
    labels = (sub["LevelRegime"] + " / " + sub["TrendRegime"]).tolist()
    vals = sub["Beta_vs_dRealYield"].values
    base_color = COLORS["blue"] if asset=="GOLD" else COLORS["purple"]

    plt.figure(figsize=(9,5))
    bars = plt.bar(labels, vals, color=base_color)
    plt.axhline(0, linestyle="--", color=COLORS["gray"])
    plt.title(f"{asset} — Beta to ΔReal Yield by Regime")
    plt.ylabel("Beta"); plt.xlabel("Regime (Level / Trend)")
    for b, v, n in zip(bars, vals, sub["Obs"].values):
        plt.text(b.get_x()+b.get_width()/2, v, f"n={int(n)}", ha="center",
                 va="bottom" if v>=0 else "top")
    plt.tight_layout()
    plt.savefig(CH / f"deepdive_5_regime_betas_{asset.lower()}.png")
    plt.close()

# Dashboard charts
safe_plot([payems], ["Nonfarm Payrolls"], [COLORS["blue"]],
          "U.S. Nonfarm Payrolls", "Thousands", "dash_1_payrolls.png")

safe_plot([unrate], ["Unemployment Rate"], [COLORS["orange"]],
          "Unemployment Rate", "%", "dash_2_unemployment.png")

safe_plot([index_100(cpi), index_100(core)],
          ["Headline CPI (idx)", "Core CPI (idx)"],
          [COLORS["blue"], COLORS["purple"]],
          "Inflation Mix — Headline vs Core (indexed)", "Index (start=100)", "dash_3_cpi_core_indexed.png")

safe_plot([lei], ["LEI"], [COLORS["gray"]],
          "Conference Board Leading Economic Index", "Index", "dash_4_lei.png")

safe_plot([umich], ["UMich Sentiment"], [COLORS["blue"]],
          "University of Michigan Consumer Sentiment", "Index", "dash_5_umich.png")

# =========================
# BLOG DRAFT (Markdown)
# =========================
today_str = date.today().strftime("%B %d, %Y")

deep_sections = [
    ("Real Yields vs. Gold (indexed)",           "deepdive_1_real_vs_gold_indexed.png",
     "Higher real yields typically weigh on gold. Lately, gold hasn’t flinched—the decoupling looks like a regime."),
    ("Real Yields vs. Trade-Weighted USD (indexed)", "deepdive_2_real_vs_usd_indexed.png",
     "The dollar’s beta to real yields looks softer than prior cycles. If that persists, the playbook changes."),
    ("12-Month Excess: Gold – USD",              "deepdive_3_gold_minus_usd_12m.png",
     "Gold’s relative carry to the dollar is positive on a 12-month lookback—more than just a hedge."),
    ("Rolling 12-Month Correlations with ΔReal Yield", "deepdive_4_rolling_corrs.png",
     "Correlations aren’t constants; they’re regimes. The gold–real yield link has weakened while USD–real yield remains directionally intact."),
    ("Regime Betas: Asset returns vs ΔReal Yield (by level/trend)", "deepdive_5_regime_betas_gold.png",
     "Gold’s sensitivity is most negative when real yields are high & rising; USD shows the mirror image."),
    ("", "deepdive_5_regime_betas_usd.png", "")
]

dash_sections = [
    ("U.S. Nonfarm Payrolls", "dash_1_payrolls.png",
     "Still expanding, but the slope is cooling—late-cycle tells."),
    ("Unemployment Rate", "dash_2_unemployment.png",
     "Edging up from the floor—small moves matter at this stage."),
    ("Inflation Mix — Headline vs Core (indexed)", "dash_3_cpi_core_indexed.png",
     "Core remains sticky; services carry the load."),
    ("Conference Board LEI", "dash_4_lei.png",
     "Still flagging slower growth; duration of weakness matters."),
    ("University of Michigan Consumer Sentiment", "dash_5_umich.png",
     "Households feel the pinch; expectations wobble more than conditions."),
]

intro = (
    "Higher real yields usually pressure gold and support the dollar. Not this time. "
    "Gold’s resilience and a softer USD beta point to a regime shift. "
    "Below is the evidence, followed by a quick macro dashboard."
)

md = []
md += [f"# {TITLE}", "", f"**{SUBTITLE}**", f"*{today_str}*", ""]
md += [intro, ""]
md += ["## Deep Dive", ""]
for head, img, cap in deep_sections:
    if head:
        md += [f"### {head}"]
    md += [f"![{head}](charts/{img})"]
    if cap:
        md += [f"*{cap}*"]
    md += [""]

md += ["## Macro Dashboard", ""]
for head, img, cap in dash_sections:
    md += [f"### {head}", f"![{head}](charts/{img})", f"*{cap}*", ""]

md += [
    "## What Would Change My Mind?", "",
    "- **Invalidate decoupling:** a sharp USD rally **and** gold drawdown on the same real-yield impulse.",
    "- **Confirm decoupling:** gold holds firm despite further grind higher in real yields.", "",
    "## Watchlist", "",
    "- Real yield trend and level (DFII10 tertiles).",
    "- USD beta drift versus Δreal yields.",
    "- Central bank gold purchases (flow support vs. cycle).", "",
    f"*Tables:* `tables/regime_betas_vs_dRealYield.csv`",
]

draft_path = OUT / "lighthouse_macro_draft.md"
with open(draft_path, "w", encoding="utf-8") as f:
    f.write("\n".join(md))

print("Draft written to:", draft_path.as_posix())
print("Charts folder:", CH.as_posix())
print("Tables folder:", TB.as_posix())
print("Note: Upload PNGs to Substack and paste the Markdown from the draft.")
```


SyntaxError: invalid syntax (ipython-input-3230268334.py, line 1)

In [ ]:
# %% [single cell] Lighthouse Macro Notebook — FRED + Alpha Vantage end-to-end
# - Pulls macro series from FRED (real yields, USD, CPI, etc.)
# - Uses Alpha Vantage (AV) as well for assets FRED doesn’t cover or as proxies (e.g., GLD, UUP, XAUUSD)
# - Builds charts in Lighthouse Macro palette
# - Computes rolling correlations and regime betas (by real yield level/trend)
# - Writes a full Markdown blog draft that references the saved charts
#
# Prereqs (run once in your environment if needed):
# pip install fredapi pandas numpy matplotlib requests statsmodels

from fredapi import Fred
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import requests
import time
from statsmodels.api import OLS, add_constant
from datetime import date

# =========================
# CONFIG & KEYS
# =========================
FRED_API_KEY = "7f8e44038ee69c4f78cf71873e85db16"                   # provided
ALPHAVANTAGE_API_KEY = "IOTZFZG01XK55BHI"                           # provided
START_DATE = "2015-01-01"                                           # adjust the analysis window
TITLE = "When Real Yields Rise but Gold Doesn’t Blink"
SUBTITLE = "A decoupling worth your attention—and a simple regime map to trade it."

# Branding palette
COLORS = {
    "blue":   "#0067db",  # primary
    "orange": "#ff8c42",  # medium-dark but bright orange
    "purple": "#6a0dad",  # deep purple
    "gray":   "#7a7a7a",  # medium-dark gray
}

plt.rcParams.update({
    "axes.edgecolor": "#333333",
    "axes.titleweight": "bold",
    "axes.labelcolor": "#333333",
    "font.size": 12,
    "figure.dpi": 150,
    "savefig.dpi": 180,
})

# Output paths
OUT = Path("lighthouse_outputs")
CH = OUT / "charts"
TB = OUT / "tables"
OUT.mkdir(exist_ok=True, parents=True)
CH.mkdir(exist_ok=True, parents=True)
TB.mkdir(exist_ok=True, parents=True)

# =========================
# HELPERS
# =========================
def monthly(series: pd.Series) -> pd.Series:
    """Coerce to month-end frequency with last available obs."""
    s = pd.Series(series).dropna()
    s.index = pd.to_datetime(s.index)
    return s.resample("M").last().ffill()

def clip(s: pd.Series, start=START_DATE) -> pd.Series:
    return s[s.index >= pd.to_datetime(start)]

def index_100(s: pd.Series) -> pd.Series:
    s = s.dropna()
    return s / s.iloc[0] * 100.0

def pct_change_12m(s: pd.Series) -> pd.Series:
    return s.pct_change(12) * 100.0

def monthly_returns(s: pd.Series) -> pd.Series:
    return s.pct_change(1).rename(s.name + "_ret")

def delta(s: pd.Series) -> pd.Series:
    """Simple first difference (e.g., monthly change in yields)."""
    return s.diff(1).rename(s.name + "_chg")

def roll_corr(a: pd.Series, b: pd.Series, window=12) -> pd.Series:
    return a.rolling(window).corr(b)

def ols_beta(y: pd.Series, x: pd.Series):
    """OLS beta of y on x with t-stat and n-obs."""
    df = pd.concat([y, x], axis=1).dropna()
    if len(df) < 12:
        return np.nan, np.nan, np.nan
    X = add_constant(df.iloc[:,1].values)
    model = OLS(df.iloc[:,0].values, X).fit()
    return float(model.params[1]), float(model.tvalues[1]), int(model.nobs)

def to_monthly_from_daily(df: pd.DataFrame, value_col: str) -> pd.Series:
    """Convert daily AV time series to monthly last."""
    s = df[value_col].copy()
    s.index = pd.to_datetime(s.index)
    s = s.sort_index().resample("M").last().ffill()
    return s

def av_get(symbol: str, function: str, **params) -> dict:
    """Generic Alpha Vantage GET with basic retry/backoff."""
    base = "https://www.alphavantage.co/query"
    payload = {"function": function, "apikey": ALPHAVANTAGE_API_KEY}
    payload.update(params)
    for i in range(3):
        r = requests.get(base, params=payload, timeout=30)
        if r.status_code == 200:
            data = r.json()
            # Handle throttling / note
            if any(k in data for k in ["Error Message", "Information", "Note"]):
                time.sleep(12 * (i + 1))
                continue
            return data
        time.sleep(3 * (i + 1))
    return {}

def av_fx_daily(from_symbol: str, to_symbol: str) -> pd.DataFrame:
    """FX daily series (e.g., XAUUSD if supported)."""
    data = av_get(function="FX_DAILY", from_symbol=from_symbol, to_symbol=to_symbol, outputsize="full")
    key = "Time Series FX (Daily)"
    if key not in data:
        return pd.DataFrame()
    df = pd.DataFrame(data[key]).T.rename(columns={
        "1. open":"open", "2. high":"high", "3. low":"low", "4. close":"close"
    }).apply(pd.to_numeric, errors="coerce")
    df.index.name = "date"
    return df

def av_equity_monthly(symbol: str) -> pd.DataFrame:
    """Monthly OHLC for ETFs/equities (e.g., GLD, UUP)"""
    data = av_get(function="TIME_SERIES_MONTHLY_ADJUSTED", symbol=symbol)
    key = "Monthly Adjusted Time Series"
    if key not in data:
        return pd.DataFrame()
    df = pd.DataFrame(data[key]).T.rename(columns={
        "1. open":"open", "2. high":"high", "3. low":"low", "4. close":"close",
        "5. adjusted close":"adj_close", "6. volume":"volume", "7. dividend amount":"dividend"
    }).apply(pd.to_numeric, errors="coerce")
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    return df

def safe_plot(series_list, labels, colors, title, ylab, fname, hline0=False):
    plt.figure(figsize=(9,5))
    for s, lab, col in zip(series_list, labels, colors):
        s.dropna().plot(label=lab, color=col)
    if hline0:
        plt.axhline(0, linestyle="--", color=COLORS["gray"])
    plt.title(title)
    plt.ylabel(ylab); plt.xlabel("")
    plt.legend()
    plt.tight_layout()
    plt.savefig(CH / fname)
    plt.close()

# =========================
# FETCH DATA — FRED first
# =========================
fred = Fred(api_key=FRED_API_KEY)

# Deep-dive backbone
dfii10 = clip(monthly(fred.get_series("DFII10")))                 # 10Y TIPS real yield (%)
usd_broad = clip(monthly(fred.get_series("DTWEXBGS")))            # Trade-Weighted USD: Broad
gold_fred = clip(monthly(fred.get_series("GOLDAMGBD228NLBM")))    # Gold (London AM USD)

# Dashboard
payems = clip(monthly(fred.get_series("PAYEMS")))                 # Nonfarm Payrolls (thousands)
unrate = clip(monthly(fred.get_series("UNRATE")))                 # Unemployment rate (%)
cpi    = clip(monthly(fred.get_series("CPIAUCSL")))               # CPI Index
core   = clip(monthly(fred.get_series("CPILFESL")))               # Core CPI Index
lei    = clip(monthly(fred.get_series("USSLIND")))                # Leading Index (Conference Board)
umich  = clip(monthly(fred.get_series("UMCSENT")))                # U. Michigan Sentiment

# =========================
# ALPHA VANTAGE — Proxies or fills
# Use GLD as market gold proxy & UUP as USD proxy (if you want market-tradable series)
# Also try XAUUSD via FX_DAILY (if AV supports the metal code in your account)
# =========================
# GLD monthly
gld_m = av_equity_monthly("GLD")
gld_m_close = gld_m["adj_close"] if not gld_m.empty else pd.Series(dtype=float)
gld_m_close.name = "GLD_adj_close"

# UUP monthly (Dollar proxy ETF)
uup_m = av_equity_monthly("UUP")
uup_m_close = uup_m["adj_close"] if not uup_m.empty else pd.Series(dtype=float)
uup_m_close.name = "UUP_adj_close"

# XAUUSD daily -> monthly (if available)
xauusd = av_fx_daily("XAU", "USD")
xauusd_m = to_monthly_from_daily(xauusd, "close") if not xauusd.empty else pd.Series(dtype=float)
xauusd_m.name = "XAUUSD"

# Trim AV proxies to START_DATE
if not gld_m_close.empty:
    gld_m_close = gld_m_close[gld_m_close.index >= pd.to_datetime(START_DATE)]
if not uup_m_close.empty:
    uup_m_close = uup_m_close[uup_m_close.index >= pd.to_datetime(START_DATE)]
if not xauusd_m.empty:
    xauusd_m = xauusd_m[xauusd_m.index >= pd.to_datetime(START_DATE)]

# =========================
# SERIES SELECTION (primary + proxies)
# =========================
gold_series = gold_fred.copy()
gold_label = "Gold (London AM, USD)"
if gold_series.dropna().empty and not gld_m_close.empty:
    gold_series = gld_m_close.copy()
    gold_label = "GLD (Adj Close, proxy)"
elif not xauusd_m.empty:
    # Option: blend or choose XAUUSD if you prefer spot-like
    # For now we keep FRED as primary; uncomment to switch:
    # gold_series = xauusd_m.copy()
    # gold_label = "XAUUSD (Alpha Vantage FX)"
    pass

usd_series = usd_broad.copy()
usd_label = "USD Broad Index (DTWEXBGS)"
if usd_series.dropna().empty and not uup_m_close.empty:
    usd_series = uup_m_close.copy()
    usd_label = "UUP (Adj Close, proxy)"

# =========================
# CALCULATIONS
# =========================
# Index for visuals
gold_idx = index_100(gold_series)
usd_idx  = index_100(usd_series)
dfii_idx = index_100(dfii10)

# Excess performance (12m)
gold_12 = pct_change_12m(gold_series)
usd_12  = pct_change_12m(usd_series)
excess  = (gold_12 - usd_12).dropna()
excess.name = "Gold minus USD (12m pp)"

# Rolling correlations (12m) with ΔReal Yield
gold_ret = monthly_returns(gold_series)
usd_ret  = monthly_returns(usd_series)
d_real   = delta(dfii10)

corr_gold_y = roll_corr(gold_ret, d_real, window=12).dropna()
corr_usd_y  = roll_corr(usd_ret,  d_real, window=12).dropna()

# Regimes: Level tertiles & 3m slope sign
p33, p67 = dfii10.quantile([0.33, 0.67])
level_regime = pd.Series(index=dfii10.index, dtype="object")
level_regime[dfii10 <= p33] = "Low"
level_regime[(dfii10 > p33) & (dfii10 <= p67)] = "Mid"
level_regime[dfii10 > p67] = "High"

def slope_3m(x: pd.Series) -> float:
    x = x.dropna()
    if len(x) < 3: return np.nan
    y = x.values
    t = np.arange(len(y))
    return np.polyfit(t, y, 1)[0]

rolling_slope = dfii10.rolling(3).apply(slope_3m, raw=False)
trend_regime = pd.Series(index=dfii10.index, dtype="object")
trend_regime[rolling_slope > 0]  = "Rising"
trend_regime[rolling_slope <= 0] = "Falling"

reg_df = pd.concat({
    "gold_ret": gold_ret,
    "usd_ret": usd_ret,
    "d_real": d_real,
    "level_regime": level_regime,
    "trend_regime": trend_regime
}, axis=1).dropna()

# Regime betas: returns ~ Δreal yield
rows = []
for asset in ["gold_ret","usd_ret"]:
    for lvl in ["Low","Mid","High"]:
        for tr in ["Falling","Rising"]:
            sub = reg_df[(reg_df["level_regime"]==lvl) & (reg_df["trend_regime"]==tr)]
            b, t, n = ols_beta(sub[asset], sub["d_real"])
            rows.append({
                "Asset": asset.replace("_ret","").upper(),
                "LevelRegime": lvl,
                "TrendRegime": tr,
                "Beta_vs_dRealYield": b,
                "t_stat": t,
                "Obs": n
            })
betas = pd.DataFrame(rows).sort_values(["Asset","LevelRegime","TrendRegime"])
betas.to_csv(TB / "regime_betas_vs_dRealYield.csv", index=False)

# Quick dashboard stats
def yoy(series):
    r = series.pct_change(12) * 100
    return float(r.dropna().iloc[-1]) if not r.dropna().empty else np.nan

dash_stats = {
    "payems_last": float(payems.dropna().iloc[-1]) if not payems.dropna().empty else np.nan,
    "unrate_last": float(unrate.dropna().iloc[-1]) if not unrate.dropna().empty else np.nan,
    "cpi_yoy": yoy(cpi),
    "core_yoy": yoy(core),
    "lei_last": float(lei.dropna().iloc[-1]) if not lei.dropna().empty else np.nan,
    "umich_last": float(umich.dropna().iloc[-1]) if not umich.dropna().empty else np.nan,
}

# =========================
# CHARTS
# =========================
safe_plot(
    [gold_idx, dfii_idx],
    [f"{gold_label} (indexed)", "10y TIPS Real Yield (indexed)"],
    [COLORS["blue"], COLORS["orange"]],
    "Real Yields vs Gold — Indexed to 100",
    "Index (start=100)",
    "deepdive_1_real_vs_gold_indexed.png"
)

safe_plot(
    [usd_idx, dfii_idx],
    [f"{usd_label} (indexed)", "10y TIPS Real Yield (indexed)"],
    [COLORS["blue"], COLORS["orange"]],
    "Real Yields vs Trade-Weighted USD — Indexed to 100",
    "Index (start=100)",
    "deepdive_2_real_vs_usd_indexed.png"
)

safe_plot(
    [excess],
    ["Gold minus USD (12m pp)"],
    [COLORS["blue"]],
    "12-Month Excess Performance: Gold minus USD",
    "pp",
    "deepdive_3_gold_minus_usd_12m.png",
    hline0=True
)

safe_plot(
    [corr_gold_y, corr_usd_y],
    ["Corr( Gold returns, ΔReal yield )", "Corr( USD returns, ΔReal yield )"],
    [COLORS["blue"], COLORS["purple"]],
    "Rolling 12-Month Correlations with ΔReal Yield",
    "Correlation",
    "deepdive_4_rolling_corrs.png",
    hline0=True
)

# Regime beta bars
for asset in ["GOLD","USD"]:
    sub = betas[betas["Asset"]==asset]
    labels = (sub["LevelRegime"] + " / " + sub["TrendRegime"]).tolist()
    vals = sub["Beta_vs_dRealYield"].values
    base_color = COLORS["blue"] if asset=="GOLD" else COLORS["purple"]

    plt.figure(figsize=(9,5))
    bars = plt.bar(labels, vals, color=base_color)
    plt.axhline(0, linestyle="--", color=COLORS["gray"])
    plt.title(f"{asset} — Beta to ΔReal Yield by Regime")
    plt.ylabel("Beta"); plt.xlabel("Regime (Level / Trend)")
    for b, v, n in zip(bars, vals, sub["Obs"].values):
        plt.text(b.get_x()+b.get_width()/2, v, f"n={int(n)}", ha="center",
                 va="bottom" if v>=0 else "top")
    plt.tight_layout()
    plt.savefig(CH / f"deepdive_5_regime_betas_{asset.lower()}.png")
    plt.close()

# Dashboard charts
safe_plot([payems], ["Nonfarm Payrolls"], [COLORS["blue"]],
          "U.S. Nonfarm Payrolls", "Thousands", "dash_1_payrolls.png")

safe_plot([unrate], ["Unemployment Rate"], [COLORS["orange"]],
          "Unemployment Rate", "%", "dash_2_unemployment.png")

safe_plot([index_100(cpi), index_100(core)],
          ["Headline CPI (idx)", "Core CPI (idx)"],
          [COLORS["blue"], COLORS["purple"]],
          "Inflation Mix — Headline vs Core (indexed)", "Index (start=100)", "dash_3_cpi_core_indexed.png")

safe_plot([lei], ["LEI"], [COLORS["gray"]],
          "Conference Board Leading Economic Index", "Index", "dash_4_lei.png")

safe_plot([umich], ["UMich Sentiment"], [COLORS["blue"]],
          "University of Michigan Consumer Sentiment", "Index", "dash_5_umich.png")

# =========================
# BLOG DRAFT (Markdown)
# =========================
today_str = date.today().strftime("%B %d, %Y")

deep_sections = [
    ("Real Yields vs. Gold (indexed)",           "deepdive_1_real_vs_gold_indexed.png",
     "Higher real yields typically weigh on gold. Lately, gold hasn’t flinched—the decoupling looks like a regime."),
    ("Real Yields vs. Trade-Weighted USD (indexed)", "deepdive_2_real_vs_usd_indexed.png",
     "The dollar’s beta to real yields looks softer than prior cycles. If that persists, the playbook changes."),
    ("12-Month Excess: Gold – USD",              "deepdive_3_gold_minus_usd_12m.png",
     "Gold’s relative carry to the dollar is positive on a 12-month lookback—more than just a hedge."),
    ("Rolling 12-Month Correlations with ΔReal Yield", "deepdive_4_rolling_corrs.png",
     "Correlations aren’t constants; they’re regimes. The gold–real yield link has weakened while USD–real yield remains directionally intact."),
    ("Regime Betas: Asset returns vs ΔReal Yield (by level/trend)", "deepdive_5_regime_betas_gold.png",
     "Gold’s sensitivity is most negative when real yields are high & rising; USD shows the mirror image."),
    ("", "deepdive_5_regime_betas_usd.png", "")
]

dash_sections = [
    ("U.S. Nonfarm Payrolls", "dash_1_payrolls.png",
     "Still expanding, but the slope is cooling—late-cycle tells."),
    ("Unemployment Rate", "dash_2_unemployment.png",
     "Edging up from the floor—small moves matter at this stage."),
    ("Inflation Mix — Headline vs Core (indexed)", "dash_3_cpi_core_indexed.png",
     "Core remains sticky; services carry the load."),
    ("Conference Board LEI", "dash_4_lei.png",
     "Still flagging slower growth; duration of weakness matters."),
    ("University of Michigan Consumer Sentiment", "dash_5_umich.png",
     "Households feel the pinch; expectations wobble more than conditions."),
]

intro = (
    "Higher real yields usually pressure gold and support the dollar. Not this time. "
    "Gold’s resilience and a softer USD beta point to a regime shift. "
    "Below is the evidence, followed by a quick macro dashboard."
)

md = []
md += [f"# {TITLE}", "", f"**{SUBTITLE}**", f"*{today_str}*", ""]
md += [intro, ""]
md += ["## Deep Dive", ""]
for head, img, cap in deep_sections:
    if head:
        md += [f"### {head}"]
    md += [f"![{head}](charts/{img})"]
    if cap:
        md += [f"*{cap}*"]
    md += [""]

md += ["## Macro Dashboard", ""]
for head, img, cap in dash_sections:
    md += [f"### {head}", f"![{head}](charts/{img})", f"*{cap}*", ""]

md += [
    "## What Would Change My Mind?", "",
    "- **Invalidate decoupling:** a sharp USD rally **and** gold drawdown on the same real-yield impulse.",
    "- **Confirm decoupling:** gold holds firm despite further grind higher in real yields.", "",
    "## Watchlist", "",
    "- Real yield trend and level (DFII10 tertiles).",
    "- USD beta drift versus Δreal yields.",
    "- Central bank gold purchases (flow support vs. cycle).", "",
    f"*Tables:* `tables/regime_betas_vs_dRealYield.csv`",
]

draft_path = OUT / "lighthouse_macro_draft.md"
with open(draft_path, "w", encoding="utf-8") as f:
    f.write("\n".join(md))

print("Draft written to:", draft_path.as_posix())
print("Charts folder:", CH.as_posix())
print("Tables folder:", TB.as_posix())
print("Note: Upload PNGs to Substack and paste the Markdown from the draft.")

In [ ]:
# Prereqs (run once in your environment if needed):
!pip install fredapi pandas numpy matplotlib requests statsmodels

In [ ]:
# %% [single cell] Lighthouse Macro Notebook — FRED + Alpha Vantage end-to-end
# - Pulls macro series from FRED (real yields, USD, CPI, etc.)
# - Uses Alpha Vantage (AV) as well for assets FRED doesn’t cover or as proxies (e.g., GLD, UUP, XAUUSD)
# - Builds charts in Lighthouse Macro palette
# - Computes rolling correlations and regime betas (by real yield level/trend)
# - Writes a full Markdown blog draft that references the saved charts
#
# Prereqs (run once in your environment if needed):
#   pip install fredapi pandas numpy matplotlib requests statsmodels

from fredapi import Fred
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import requests
import time
from statsmodels.api import OLS, add_constant
from datetime import date

# =========================
# CONFIG & KEYS
# =========================
FRED_API_KEY = "7f8e44038ee69c4f78cf71873e85db16"                   # provided
ALPHAVANTAGE_API_KEY = "IOTZFZG01XK55BHI"                           # provided
START_DATE = "2015-01-01"                                           # adjust the analysis window
TITLE = "When Real Yields Rise but Gold Doesn’t Blink"
SUBTITLE = "A decoupling worth your attention—and a simple regime map to trade it."

# Branding palette
COLORS = {
    "blue":   "#0067db",  # primary
    "orange": "#ff8c42",  # medium-dark but bright orange
    "purple": "#6a0dad",  # deep purple
    "gray":   "#7a7a7a",  # medium-dark gray
}

plt.rcParams.update({
    "axes.edgecolor": "#333333",
    "axes.titleweight": "bold",
    "axes.labelcolor": "#333333",
    "font.size": 12,
    "figure.dpi": 150,
    "savefig.dpi": 180,
})

# Output paths
OUT = Path("lighthouse_outputs")
CH = OUT / "charts"
TB = OUT / "tables"
OUT.mkdir(exist_ok=True, parents=True)
CH.mkdir(exist_ok=True, parents=True)
TB.mkdir(exist_ok=True, parents=True)

# =========================
# HELPERS
# =========================
def monthly(series: pd.Series) -> pd.Series:
    """Coerce to month-end frequency with last available obs."""
    s = pd.Series(series).dropna()
    s.index = pd.to_datetime(s.index)
    return s.resample("M").last().ffill()

def clip(s: pd.Series, start=START_DATE) -> pd.Series:
    return s[s.index >= pd.to_datetime(start)]

def index_100(s: pd.Series) -> pd.Series:
    s = s.dropna()
    return s / s.iloc[0] * 100.0

def pct_change_12m(s: pd.Series) -> pd.Series:
    return s.pct_change(12) * 100.0

def monthly_returns(s: pd.Series) -> pd.Series:
    return s.pct_change(1).rename(s.name + "_ret")

def delta(s: pd.Series) -> pd.Series:
    """Simple first difference (e.g., monthly change in yields)."""
    return s.diff(1).rename(s.name + "_chg")

def roll_corr(a: pd.Series, b: pd.Series, window=12) -> pd.Series:
    return a.rolling(window).corr(b)

def ols_beta(y: pd.Series, x: pd.Series):
    """OLS beta of y on x with t-stat and n-obs."""
    df = pd.concat([y, x], axis=1).dropna()
    if len(df) < 12:
        return np.nan, np.nan, np.nan
    X = add_constant(df.iloc[:,1].values)
    model = OLS(df.iloc[:,0].values, X).fit()
    return float(model.params[1]), float(model.tvalues[1]), int(model.nobs)

def to_monthly_from_daily(df: pd.DataFrame, value_col: str) -> pd.Series:
    """Convert daily AV time series to monthly last."""
    s = df[value_col].copy()
    s.index = pd.to_datetime(s.index)
    s = s.sort_index().resample("M").last().ffill()
    return s

def av_get(symbol: str, function: str, **params) -> dict:
    """Generic Alpha Vantage GET with basic retry/backoff."""
    base = "https://www.alphavantage.co/query"
    payload = {"function": function, "apikey": ALPHAVANTAGE_API_KEY}
    payload.update(params)
    for i in range(3):
        r = requests.get(base, params=payload, timeout=30)
        if r.status_code == 200:
            data = r.json()
            # Handle throttling / note
            if any(k in data for k in ["Error Message", "Information", "Note"]):
                time.sleep(12 * (i + 1))
                continue
            return data
        time.sleep(3 * (i + 1))
    return {}

def av_fx_daily(from_symbol: str, to_symbol: str) -> pd.DataFrame:
    """FX daily series (e.g., XAUUSD if supported)."""
    data = av_get(function="FX_DAILY", from_symbol=from_symbol, to_symbol=to_symbol, outputsize="full")
    key = "Time Series FX (Daily)"
    if key not in data:
        return pd.DataFrame()
    df = pd.DataFrame(data[key]).T.rename(columns={
        "1. open":"open", "2. high":"high", "3. low":"low", "4. close":"close"
    }).apply(pd.to_numeric, errors="coerce")
    df.index.name = "date"
    return df

def av_equity_monthly(symbol: str) -> pd.DataFrame:
    """Monthly OHLC for ETFs/equities (e.g., GLD, UUP)"""
    data = av_get(function="TIME_SERIES_MONTHLY_ADJUSTED", symbol=symbol)
    key = "Monthly Adjusted Time Series"
    if key not in data:
        return pd.DataFrame()
    df = pd.DataFrame(data[key]).T.rename(columns={
        "1. open":"open", "2. high":"high", "3. low":"low", "4. close":"close",
        "5. adjusted close":"adj_close", "6. volume":"volume", "7. dividend amount":"dividend"
    }).apply(pd.to_numeric, errors="coerce")
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    return df

def safe_plot(series_list, labels, colors, title, ylab, fname, hline0=False):
    plt.figure(figsize=(9,5))
    for s, lab, col in zip(series_list, labels, colors):
        s.dropna().plot(label=lab, color=col)
    if hline0:
        plt.axhline(0, linestyle="--", color=COLORS["gray"])
    plt.title(title)
    plt.ylabel(ylab); plt.xlabel("")
    plt.legend()
    plt.tight_layout()
    plt.savefig(CH / fname)
    plt.close()

# =========================
# FETCH DATA — FRED first
# =========================
fred = Fred(api_key=FRED_API_KEY)

# Deep-dive backbone
dfii10 = clip(monthly(fred.get_series("DFII10")))                 # 10Y TIPS real yield (%)
usd_broad = clip(monthly(fred.get_series("DTWEXBGS")))            # Trade-Weighted USD: Broad
gold_fred = clip(monthly(fred.get_series("GOLDAMGBD228NLBM")))    # Gold (London AM USD)

# Dashboard
payems = clip(monthly(fred.get_series("PAYEMS")))                 # Nonfarm Payrolls (thousands)
unrate = clip(monthly(fred.get_series("UNRATE")))                 # Unemployment rate (%)
cpi    = clip(monthly(fred.get_series("CPIAUCSL")))               # CPI Index
core   = clip(monthly(fred.get_series("CPILFESL")))               # Core CPI Index
lei    = clip(monthly(fred.get_series("USSLIND")))                # Leading Index (Conference Board)
umich  = clip(monthly(fred.get_series("UMCSENT")))                # U. Michigan Sentiment

# =========================
# ALPHA VANTAGE — Proxies or fills
# Use GLD as market gold proxy & UUP as USD proxy (if you want market-tradable series)
# Also try XAUUSD via FX_DAILY (if AV supports the metal code in your account)
# =========================
# GLD monthly
gld_m = av_equity_monthly("GLD")
gld_m_close = gld_m["adj_close"] if not gld_m.empty else pd.Series(dtype=float)
gld_m_close.name = "GLD_adj_close"

# UUP monthly (Dollar proxy ETF)
uup_m = av_equity_monthly("UUP")
uup_m_close = uup_m["adj_close"] if not uup_m.empty else pd.Series(dtype=float)
uup_m_close.name = "UUP_adj_close"

# XAUUSD daily -> monthly (if available)
xauusd = av_fx_daily("XAU", "USD")
xauusd_m = to_monthly_from_daily(xauusd, "close") if not xauusd.empty else pd.Series(dtype=float)
xauusd_m.name = "XAUUSD"

# Trim AV proxies to START_DATE
if not gld_m_close.empty:
    gld_m_close = gld_m_close[gld_m_close.index >= pd.to_datetime(START_DATE)]
if not uup_m_close.empty:
    uup_m_close = uup_m_close[uup_m_close.index >= pd.to_datetime(START_DATE)]
if not xauusd_m.empty:
    xauusd_m = xauusd_m[xauusd_m.index >= pd.to_datetime(START_DATE)]

# =========================
# SERIES SELECTION (primary + proxies)
# =========================
gold_series = gold_fred.copy()
gold_label = "Gold (London AM, USD)"
if gold_series.dropna().empty and not gld_m_close.empty:
    gold_series = gld_m_close.copy()
    gold_label = "GLD (Adj Close, proxy)"
elif not xauusd_m.empty:
    # Option: blend or choose XAUUSD if you prefer spot-like
    # For now we keep FRED as primary; uncomment to switch:
    # gold_series = xauusd_m.copy()
    # gold_label = "XAUUSD (Alpha Vantage FX)"
    pass

usd_series = usd_broad.copy()
usd_label = "USD Broad Index (DTWEXBGS)"
if usd_series.dropna().empty and not uup_m_close.empty:
    usd_series = uup_m_close.copy()
    usd_label = "UUP (Adj Close, proxy)"

# =========================
# CALCULATIONS
# =========================
# Index for visuals
gold_idx = index_100(gold_series)
usd_idx  = index_100(usd_series)
dfii_idx = index_100(dfii10)

# Excess performance (12m)
gold_12 = pct_change_12m(gold_series)
usd_12  = pct_change_12m(usd_series)
excess  = (gold_12 - usd_12).dropna()
excess.name = "Gold minus USD (12m pp)"

# Rolling correlations (12m) with ΔReal Yield
gold_ret = monthly_returns(gold_series)
usd_ret  = monthly_returns(usd_series)
d_real   = delta(dfii10)

corr_gold_y = roll_corr(gold_ret, d_real, window=12).dropna()
corr_usd_y  = roll_corr(usd_ret,  d_real, window=12).dropna()

# Regimes: Level tertiles & 3m slope sign
p33, p67 = dfii10.quantile([0.33, 0.67])
level_regime = pd.Series(index=dfii10.index, dtype="object")
level_regime[dfii10 <= p33] = "Low"
level_regime[(dfii10 > p33) & (dfii10 <= p67)] = "Mid"
level_regime[dfii10 > p67] = "High"

def slope_3m(x: pd.Series) -> float:
    x = x.dropna()
    if len(x) < 3: return np.nan
    y = x.values
    t = np.arange(len(y))
    return np.polyfit(t, y, 1)[0]

rolling_slope = dfii10.rolling(3).apply(slope_3m, raw=False)
trend_regime = pd.Series(index=dfii10.index, dtype="object")
trend_regime[rolling_slope > 0]  = "Rising"
trend_regime[rolling_slope <= 0] = "Falling"

reg_df = pd.concat({
    "gold_ret": gold_ret,
    "usd_ret": usd_ret,
    "d_real": d_real,
    "level_regime": level_regime,
    "trend_regime": trend_regime
}, axis=1).dropna()

# Regime betas: returns ~ Δreal yield
rows = []
for asset in ["gold_ret","usd_ret"]:
    for lvl in ["Low","Mid","High"]:
        for tr in ["Falling","Rising"]:
            sub = reg_df[(reg_df["level_regime"]==lvl) & (reg_df["trend_regime"]==tr)]
            b, t, n = ols_beta(sub[asset], sub["d_real"])
            rows.append({
                "Asset": asset.replace("_ret","").upper(),
                "LevelRegime": lvl,
                "TrendRegime": tr,
                "Beta_vs_dRealYield": b,
                "t_stat": t,
                "Obs": n
            })
betas = pd.DataFrame(rows).sort_values(["Asset","LevelRegime","TrendRegime"])
betas.to_csv(TB / "regime_betas_vs_dRealYield.csv", index=False)

# Quick dashboard stats
def yoy(series):
    r = series.pct_change(12) * 100
    return float(r.dropna().iloc[-1]) if not r.dropna().empty else np.nan

dash_stats = {
    "payems_last": float(payems.dropna().iloc[-1]) if not payems.dropna().empty else np.nan,
    "unrate_last": float(unrate.dropna().iloc[-1]) if not unrate.dropna().empty else np.nan,
    "cpi_yoy": yoy(cpi),
    "core_yoy": yoy(core),
    "lei_last": float(lei.dropna().iloc[-1]) if not lei.dropna().empty else np.nan,
    "umich_last": float(umich.dropna().iloc[-1]) if not umich.dropna().empty else np.nan,
}

# =========================
# CHARTS
# =========================
safe_plot(
    [gold_idx, dfii_idx],
    [f"{gold_label} (indexed)", "10y TIPS Real Yield (indexed)"],
    [COLORS["blue"], COLORS["orange"]],
    "Real Yields vs Gold — Indexed to 100",
    "Index (start=100)",
    "deepdive_1_real_vs_gold_indexed.png"
)

safe_plot(
    [usd_idx, dfii_idx],
    [f"{usd_label} (indexed)", "10y TIPS Real Yield (indexed)"],
    [COLORS["blue"], COLORS["orange"]],
    "Real Yields vs Trade-Weighted USD — Indexed to 100",
    "Index (start=100)",
    "deepdive_2_real_vs_usd_indexed.png"
)

safe_plot(
    [excess],
    ["Gold minus USD (12m pp)"],
    [COLORS["blue"]],
    "12-Month Excess Performance: Gold minus USD",
    "pp",
    "deepdive_3_gold_minus_usd_12m.png",
    hline0=True
)

safe_plot(
    [corr_gold_y, corr_usd_y],
    ["Corr( Gold returns, ΔReal yield )", "Corr( USD returns, ΔReal yield )"],
    [COLORS["blue"], COLORS["purple"]],
    "Rolling 12-Month Correlations with ΔReal Yield",
    "Correlation",
    "deepdive_4_rolling_corrs.png",
    hline0=True
)

# Regime beta bars
for asset in ["GOLD","USD"]:
    sub = betas[betas["Asset"]==asset]
    labels = (sub["LevelRegime"] + " / " + sub["TrendRegime"]).tolist()
    vals = sub["Beta_vs_dRealYield"].values
    base_color = COLORS["blue"] if asset=="GOLD" else COLORS["purple"]

    plt.figure(figsize=(9,5))
    bars = plt.bar(labels, vals, color=base_color)
    plt.axhline(0, linestyle="--", color=COLORS["gray"])
    plt.title(f"{asset} — Beta to ΔReal Yield by Regime")
    plt.ylabel("Beta"); plt.xlabel("Regime (Level / Trend)")
    for b, v, n in zip(bars, vals, sub["Obs"].values):
        plt.text(b.get_x()+b.get_width()/2, v, f"n={int(n)}", ha="center",
                 va="bottom" if v>=0 else "top")
    plt.tight_layout()
    plt.savefig(CH / f"deepdive_5_regime_betas_{asset.lower()}.png")
    plt.close()

# Dashboard charts
safe_plot([payems], ["Nonfarm Payrolls"], [COLORS["blue"]],
          "U.S. Nonfarm Payrolls", "Thousands", "dash_1_payrolls.png")

safe_plot([unrate], ["Unemployment Rate"], [COLORS["orange"]],
          "Unemployment Rate", "%", "dash_2_unemployment.png")

safe_plot([index_100(cpi), index_100(core)],
          ["Headline CPI (idx)", "Core CPI (idx)"],
          [COLORS["blue"], COLORS["purple"]],
          "Inflation Mix — Headline vs Core (indexed)", "Index (start=100)", "dash_3_cpi_core_indexed.png")

safe_plot([lei], ["LEI"], [COLORS["gray"]],
          "Conference Board Leading Economic Index", "Index", "dash_4_lei.png")

safe_plot([umich], ["UMich Sentiment"], [COLORS["blue"]],
          "University of Michigan Consumer Sentiment", "Index", "dash_5_umich.png")

# =========================
# BLOG DRAFT (Markdown)
# =========================
today_str = date.today().strftime("%B %d, %Y")

deep_sections = [
    ("Real Yields vs. Gold (indexed)",           "deepdive_1_real_vs_gold_indexed.png",
     "Higher real yields typically weigh on gold. Lately, gold hasn’t flinched—the decoupling looks like a regime."),
    ("Real Yields vs. Trade-Weighted USD (indexed)", "deepdive_2_real_vs_usd_indexed.png",
     "The dollar’s beta to real yields looks softer than prior cycles. If that persists, the playbook changes."),
    ("12-Month Excess: Gold – USD",              "deepdive_3_gold_minus_usd_12m.png",
     "Gold’s relative carry to the dollar is positive on a 12-month lookback—more than just a hedge."),
    ("Rolling 12-Month Correlations with ΔReal Yield", "deepdive_4_rolling_corrs.png",
     "Correlations aren’t constants; they’re regimes. The gold–real yield link has weakened while USD–real yield remains directionally intact."),
    ("Regime Betas: Asset returns vs ΔReal Yield (by level/trend)", "deepdive_5_regime_betas_gold.png",
     "Gold’s sensitivity is most negative when real yields are high & rising; USD shows the mirror image."),
    ("", "deepdive_5_regime_betas_usd.png", "")
]

dash_sections = [
    ("U.S. Nonfarm Payrolls", "dash_1_payrolls.png",
     "Still expanding, but the slope is cooling—late-cycle tells."),
    ("Unemployment Rate", "dash_2_unemployment.png",
     "Edging up from the floor—small moves matter at this stage."),
    ("Inflation Mix — Headline vs Core (indexed)", "dash_3_cpi_core_indexed.png",
     "Core remains sticky; services carry the load."),
    ("Conference Board LEI", "dash_4_lei.png",
     "Still flagging slower growth; duration of weakness matters."),
    ("University of Michigan Consumer Sentiment", "dash_5_umich.png",
     "Households feel the pinch; expectations wobble more than conditions."),
]

intro = (
    "Higher real yields usually pressure gold and support the dollar. Not this time. "
    "Gold’s resilience and a softer USD beta point to a regime shift. "
    "Below is the evidence, followed by a quick macro dashboard."
)

md = []
md += [f"# {TITLE}", "", f"**{SUBTITLE}**", f"*{today_str}*", ""]
md += [intro, ""]
md += ["## Deep Dive", ""]
for head, img, cap in deep_sections:
    if head:
        md += [f"### {head}"]
    md += [f"![{head}](charts/{img})"]
    if cap:
        md += [f"*{cap}*"]
    md += [""]

md += ["## Macro Dashboard", ""]
for head, img, cap in dash_sections:
    md += [f"### {head}", f"![{head}](charts/{img})", f"*{cap}*", ""]

md += [
    "## What Would Change My Mind?", "",
    "- **Invalidate decoupling:** a sharp USD rally **and** gold drawdown on the same real-yield impulse.",
    "- **Confirm decoupling:** gold holds firm despite further grind higher in real yields.", "",
    "## Watchlist", "",
    "- Real yield trend and level (DFII10 tertiles).",
    "- USD beta drift versus Δreal yields.",
    "- Central bank gold purchases (flow support vs. cycle).", "",
    f"*Tables:* `tables/regime_betas_vs_dRealYield.csv`",
]

draft_path = OUT / "lighthouse_macro_draft.md"
with open(draft_path, "w", encoding="utf-8") as f:
    f.write("\n".join(md))

print("Draft written to:", draft_path.as_posix())
print("Charts folder:", CH.as_posix())
print("Tables folder:", TB.as_posix())
print("Note: Upload PNGs to Substack and paste the Markdown from the draft.")


/tmp/ipython-input-1562871327.py:62: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  return s.resample("ME").last().ffill()
/tmp/ipython-input-1562871327.py:62: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  return s.resample("ME").last().ffill()


ValueError: Bad Request.  The series does not exist.